In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.nn.modules.loss import NLLLoss
import shutil
import sys
sys.path.append("../..") # Path way to library root

/home/alan/miniconda3/envs/ml_gpu/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/alan/miniconda3/envs/ml_gpu/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
!which python

/home/alan/miniconda3/envs/ml_gpu/bin/python


In [3]:

from ml_framework.modeling.architectures.mlp import MLP

In [4]:
loss_function = NLLLoss()

In [5]:
use_cuda = torch.cuda.is_available()
use_cuda

True

In [32]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

batch_size = 64
test_batch_size = 64
lr = 0.01
gamma = 0.5

train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                   'pin_memory': True,
                   'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    torch.flatten
    ])
dataset1 = datasets.MNIST(
    './data',
    train=True,
    download=True,
    transform=transform)
dataset2 = datasets.MNIST(
    './data',
    train=False,
    transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)
model = MLP(
    784,
    10,
    latent_sizes=[10, 10],
    activation_names=["ReLU", "ReLU"],
    activation_parameters= [[], []],
    dropout_levels=[0.02, 0.02],
)
optimizer = optim.Adadelta(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=1, gamma=gamma)


In [33]:
from ml_framework.training.trainer import Trainer
Trainer().train(
    model=model,
    train_dataloader=train_loader,
    test_dataloader=test_loader,
    optimizer=optimizer,
    loss_function=loss_function,
    epochs=10,
    steps_per_test=100,
    learning_rate_scheduler=scheduler,
)

In [10]:
shutil.rmtree('data')